In [1]:
# 验证JW-coding的过程
# https://learn.microsoft.com/en-us/azure/quantum/user-guide/libraries/chemistry/concepts/jordan-wigner

In [1]:
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import qchem

import numpy as np
import qiskit
from qiskit import QuantumCircuit

In [32]:
from functools import reduce

# eg. Dag(|a>) = <a|
Dag = lambda matrix: matrix.conj().T
# eg. Kron(I, X, Y) = I ⊗ X ⊗ Y，计算张量用
Kron = lambda *matrices: reduce(np.kron, matrices)
# eg. Multi(I, X, Y) = I @ X @ Y，计算矩阵连乘
Multi = lambda *matrices: reduce(np.matmul, matrices)

In [3]:
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.375, 0.0, 0.0, 0.375])

In [4]:
I = np.eye(2)

# pauli matrixes
X = np.matrix([
    [0, 1], [1, 0]
])
Y = np.matrix([
    [0, -1j], [1j, 0]
])
Z = np.matrix([
    [1, 0], [0, -1]
])

In [7]:
# 参考的JW编码后的哈密顿量
Hami_JW = \
    -0.81530 * Kron(I, I, I, I) +\
    0.16988  * Kron(Z, I, I, I) +\
    0.16988  * Kron(I, Z, I, I) +\
    0.16821  * Kron(Z, Z, I, I) +\
    0.04544  * Kron(Y, X, X, Y) +\
    -0.04544 * Kron(Y, Y, X, X) +\
    -0.04544 * Kron(X, X, Y, Y) +\
    0.04544  * Kron(X, Y, Y, X) +\
    -0.21886 * Kron(I, I, Z, I) +\
    0.12005  * Kron(Z, I, Z, I) +\
    -0.21886 * Kron(I, I, I, Z) +\
    0.16549  * Kron(Z, I, I, Z) +\
    0.16549  * Kron(I, Z, Z, I) +\
    0.12005  * Kron(I, Z, I, Z) +\
    0.17395  * Kron(I, I, Z, Z)

In [8]:
def a(i, n):
    arr = []
    for idx in range(n):
        if idx < i:
            arr.append(Z)
        elif idx == i:
            arr.append((X + 1j * Y) / 2)
        else:
            arr.append(I)
    
    return Kron(*arr)

In [11]:
def adg(i, n):
    arr = []
    for idx in range(n):
        if idx < i:
            arr.append(Z)
        elif idx == i:
            arr.append((X - 1j * Y) / 2)
        else:
            arr.append(I)
    
    return Kron(*arr)

In [75]:
sq_form_arr = [
    { 'coef': -1.24728, 'index': [0, 0] },
    { 'coef': -1.24728, 'index': [1, 1] },
    { 'coef': -0.48127, 'index': [2, 2] },
    { 'coef': -0.48127, 'index': [3, 3] },
    
    { 'coef': -0.67284, 'index': [1, 0, 1, 0] },
    { 'coef': -0.18177, 'index': [1, 0, 3, 2] },
    { 'coef': -0.48020, 'index': [2, 0, 2, 0] },
    { 'coef': -0.66197, 'index': [2, 1, 2, 1] },
    { 'coef': 0.18177, 'index': [2, 1, 3, 0] },
    { 'coef': 0.18177, 'index': [3, 0, 2, 1] },
    { 'coef':-0.66197, 'index': [3, 0, 3, 0] },
    { 'coef': -0.18177, 'index': [3, 2, 1, 0] },
    { 'coef': -0.69581, 'index': [3, 2, 3, 2] },
    { 'coef': -0.48020, 'index': [3, 1, 3, 1] },
]

In [79]:
def JW_encode(sq_form_arr, n_qubits):
    H = (np.eye(2 ** n_qubits) - np.eye(2 ** n_qubits)).astype('complex128')
    
    for sq_form_unit in sq_form_arr:
        arr = []
        index = sq_form_unit['index']
        
        for i in range(len(index) // 2):
            arr.append(adg(index[i], n_qubits))
        for i in range(len(index) // 2, len(index)):
            arr.append(a(index[i], n_qubits))
        
        H += sq_form_unit['coef'] * Multi(*arr)
        
    return H

In [80]:
Hami = JW_encode(sq_form_arr, 4)

In [81]:
np.allclose(Hami, Hami_JW, atol=1e-4)

True